In [34]:
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import os
import feather
import math
# from fastai.imports import *
# from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [3]:
data = pd.read_csv('Train.csv')
pd.options.display.max_columns = None  #added to show all the columns inplace of the dots.
# pd.options.display.max_columns = None  #added to show all the columns inplace of the dots.
data.head(10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,23.5,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,226,226,NaN,NaN,NaN,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,PC120-6E,PC120,NaN,-6E,NaN,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,TEX,Track Excavators,NaN,EROPS w AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,S175,S175,NaN,NaN,NaN,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,EROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1139255,26500,1001274,4605,121,3.0,2004,508.0,Low,12/18/2008 0:00,310G,310,G,NaN,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,Arizona,BL,Backhoe Loaders,Four Wheel Drive,OROPS,None or Unspecified,None or Unspecified,No,Extended,Powershuttle,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1139256,21000,772701,1937,121,3.0,1993,11540.0,High,8/26/2004 0:00,790ELC,790,E,NaN,LC,Large / Medium,"Hydraulic Excavator, Track - 21.0 to 24.0 Metr...",Florida,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
7,1139261,27000,902002,3539,121,3.0,2001,4883.0,High,11/17/2005 0:00,416D,416,D,NaN,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,Illinois,BL,Backhoe Loaders,Four Wheel Drive,OROPS,None or Unspecified,Reversible,No,Standard,Standard,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1139272,21500,1036251,36003,121,3.0,2008,302.0,Low,8/27/2009 0:00,430HAG,430,HAG,NaN,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Texas,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,Manual,NaN,NaN,NaN,Rubber,None or Unspecified,None or Unspecified,None or Unspecif

In [4]:
# data.info(null_counts=True)  #to check how many values are null in the dataframe. So that we can remove if a feature is mostly nan.
# data.state.value_counts() #to check what values are present in particular feature.

In [5]:
#Dropping the features that are lesser than "1 lakh" non-null values and features that I think are not providing much information.

drop_list = ['Coupler_System','Grouser_Tracks','Hydraulics_Flow','Track_Type','Undercarriage_Pad_Width','Undercarriage_Pad_Width','Stick_Length','Thumb','Pattern_Changer','Grouser_Type','Backhoe_Mounting','Blade_Type','Travel_Controls','Differential_Type','Steering_Controls',
             'UsageBand','fiModelSeries','fiModelDescriptor','Pad_Type','Stick','Turbocharged','Blade_Extension','Blade_Width','Enclosure_Type','Engine_Horsepower','Pushblock','Ripper','Scarifier','Tip_Control','Tire_Size','Forks','Coupler','UsageBand','fiModelSeries','fiModelDescriptor','Pad_Type','Stick','Turbocharged','Blade_Extension','Blade_Width','Enclosure_Type','Engine_Horsepower','Pushblock','Ripper','Scarifier','Tip_Control','Tire_Size','Forks','Coupler','Drive_System','Ride_Control']

data = data.drop(labels=drop_list,axis=1)
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 20 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
saledate                    401125 non-null object
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                       401125 non-null object
ProductGroup                401125 non-null object
ProductGroupDesc            401125 non-null object
Enclosure                 

In [6]:
#encoding the 50 states of USA.

states = ['Florida','Texas','California','Washington','Georgia','Maryland','Mississippi','Ohio','Colorado','Illinois','New Jersey','North Carolina','Tennessee','Alabama'
         'Pennsylvania','South Carolina','Arizona','New York','Connecticut','Minnesota','Missouri','Nevada','Louisiana','Kentucky','Maine','Indiana','Arkansas','New Mexico','Utah','Unspecified','New Hampshire','Wisconsin','Virginia','Idaho','Oregon'
         'Michigan','Wyoming','Montana','Oklahoma','Iowa','West Virginia','Nebraska','Kansas','Delaware','Alaska','North Dakota','Massachusetts','Vermont','South Dakota','Hawaii','Rhode Island','Puerto Rico','Washington DC']
ordered_states = {}

for i, name in enumerate(states):
    ordered_states[name] = i
print(ordered_states)


{'Florida': 0, 'Texas': 1, 'California': 2, 'Washington': 3, 'Georgia': 4, 'Maryland': 5, 'Mississippi': 6, 'Ohio': 7, 'Colorado': 8, 'Illinois': 9, 'New Jersey': 10, 'North Carolina': 11, 'Tennessee': 12, 'AlabamaPennsylvania': 13, 'South Carolina': 14, 'Arizona': 15, 'New York': 16, 'Connecticut': 17, 'Minnesota': 18, 'Missouri': 19, 'Nevada': 20, 'Louisiana': 21, 'Kentucky': 22, 'Maine': 23, 'Indiana': 24, 'Arkansas': 25, 'New Mexico': 26, 'Utah': 27, 'Unspecified': 28, 'New Hampshire': 29, 'Wisconsin': 30, 'Virginia': 31, 'Idaho': 32, 'OregonMichigan': 33, 'Wyoming': 34, 'Montana': 35, 'Oklahoma': 36, 'Iowa': 37, 'West Virginia': 38, 'Nebraska': 39, 'Kansas': 40, 'Delaware': 41, 'Alaska': 42, 'North Dakota': 43, 'Massachusetts': 44, 'Vermont': 45, 'South Dakota': 46, 'Hawaii': 47, 'Rhode Island': 48, 'Puerto Rico': 49, 'Washington DC': 50}


In [7]:
data['state'] = data['state'].map(ordered_states)
data['state'] = data['state'].fillna(51)
data['state'] = data['state'].astype(int)

data.state.head(10)


data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 20 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
saledate                    401125 non-null object
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                       401125 non-null int64
ProductGroup                401125 non-null object
ProductGroupDesc            401125 non-null object
Enclosure                  

In [8]:
#encoding Transmission.

#this encoder works fine.
#we are using .astype(str) since pandas data type 'Object' indicates mixed types rather than str type

le = preprocessing.LabelEncoder()
data.Transmission.value_counts()
data['Transmission'] = le.fit_transform(data['Transmission'].astype(str))
data.Transmission.value_counts()


8    217895
7    140328
4     23147
5     11731
6      4244
3      3204
2       418
1       114
0        44
Name: Transmission, dtype: int64

In [9]:
#encoding Hydraulics
data['Hydraulics'] = le.fit_transform(data['Hydraulics'].astype(str))
data.Hydraulics.value_counts()

0     141404
11    104423
12     80555
3      40737
4      24770
1       5622
2       2960
6        299
5        127
8         89
7         76
9         53
10        10
Name: Hydraulics, dtype: int64

In [10]:
#Encoding Enclosure

data['Enclosure'] = le.fit_transform(data['Enclosure'].astype(str))
data.Enclosure.value_counts()

5    173932
0    139026
2     87820
6       325
1        17
3         3
4         2
Name: Enclosure, dtype: int64

In [11]:
#encoding ProductGroup

data['ProductGroup'] = le.fit_transform(data['ProductGroup'].astype(str))
data.ProductGroup.value_counts()

3    101167
4     80520
0     79415
5     71046
2     43488
1     25489
Name: ProductGroup, dtype: int64

In [12]:
# Encoding ProductGroupDesc
data['ProductGroupDesc'] = le.fit_transform(data['ProductGroupDesc'].astype(str))
data.ProductGroupDesc.value_counts()

3    101167
4     80520
0     79415
5     71046
2     43488
1     25489
Name: ProductGroupDesc, dtype: int64

In [13]:
# Encoding fiModelDesc
data['fiModelDesc'] = le.fit_transform(data['fiModelDesc'].astype(str))
data.fiModelDesc.value_counts()

492     5039
737     4869
1056    4315
490     4233
105     4083
735     3718
1059    3442
489     3402
81      3216
1075    3139
1394    2848
187     2629
1746    2486
252     2453
746     2434
2364    2405
1046    2379
1718    2266
1408    2066
2363    2010
734     1998
2276    1915
2273    1851
1764    1839
503     1823
316     1794
1015    1719
546     1685
183     1603
107     1596
        ... 
4699       1
1339       1
3360       1
1083       1
1057       1
3815       1
4899       1
2498       1
3507       1
2672       1
946        1
2928       1
4071       1
690        1
4979       1
2754       1
4784       1
3010       1
3873       1
3696       1
2618       1
3799       1
1649       1
2483       1
4921       1
743        1
827        1
2227       1
3543       1
3949       1
Name: fiModelDesc, Length: 4999, dtype: int64

In [14]:
# Encoding fiBaseModel

data['fiBaseModel'] = le.fit_transform(data['fiBaseModel'].astype(str))
data.fiBaseModel.value_counts()

323     19798
174     17354
703     13110
242     12687
695      9342
526      7406
680      6826
715      6733
689      6464
18       6143
29       5809
532      5798
185      5529
304      4593
239      4398
195      4229
351      4190
264      4015
538      3907
710      3684
176      3327
310      3318
191      3272
92       3128
1387     3074
19       3022
348      3022
405      2986
63       2794
107      2775
        ...  
1820        1
840         1
73          1
964         1
1354        1
1512        1
843         1
451         1
489         1
962         1
333         1
1473        1
1613        1
590         1
1306        1
1870        1
335         1
591         1
1616        1
1872        1
1514        1
1618        1
1107        1
1363        1
340         1
1725        1
1469        1
1515        1
1877        1
1006        1
Name: fiBaseModel, Length: 1950, dtype: int64

In [15]:
# Encoding fiSecondaryDesc

data['fiSecondaryDesc'] = le.fit_transform(data['fiSecondaryDesc'].astype(str))
data.fiSecondaryDesc.value_counts()

175    137191
28      43235
20      39203
56      36424
58      24050
47      21048
40      19457
54       9153
74       7779
75       5508
14       5504
90       5163
72       4370
106      4203
76       4027
134      3853
100      3625
114      3327
84       2459
166      2111
167      1740
136      1647
112      1559
119      1369
156      1315
121      1289
53        995
133       903
152       838
150       820
        ...  
77          1
107         1
81          1
92          1
71          1
69          1
12          1
68          1
83          1
66          1
104         1
95          1
138         1
41          1
48          1
36          1
35          1
34          1
33          1
91          1
129         1
52          1
59          1
146         1
25          1
120         1
116         1
115         1
153         1
0           1
Name: fiSecondaryDesc, Length: 176, dtype: int64

In [16]:
# Encoding ProductSize

data['ProductSize'] = le.fit_transform(data['ProductSize'].astype(str))
data.ProductSize.value_counts()

6    210775
3     62274
2     49678
5     26494
4     24840
1     20975
0      6089
Name: ProductSize, dtype: int64

In [17]:
# Encoding fiProductClassDesc

data['fiProductClassDesc'] = le.fit_transform(data['fiProductClassDesc'].astype(str))
data.fiProductClassDesc.value_counts()

1     56166
50    17788
61    15114
53    14731
13    13323
7     11354
47    11140
51    10980
2     10566
59    10551
38     9321
39     9011
31     8866
18     8715
40     7565
48     7561
62     7471
37     7361
24     7317
56     7105
34     7096
52     6773
21     6608
49     6557
11     6529
14     6235
15     6134
16     5860
10     5688
30     5487
      ...  
26     2640
42     2608
19     2537
6      2258
5      2147
22     1975
23     1702
65     1476
64     1324
33     1173
36     1155
58     1041
68      764
44      741
45      739
43      557
73      438
71      432
69      389
25      348
55      224
20      135
28      117
54      113
27       62
29       38
57       25
9        21
17        1
35        1
Name: fiProductClassDesc, Length: 74, dtype: int64

In [18]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 20 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
saledate                    401125 non-null object
fiModelDesc                 401125 non-null int64
fiBaseModel                 401125 non-null int64
fiSecondaryDesc             401125 non-null int64
ProductSize                 401125 non-null int64
fiProductClassDesc          401125 non-null int64
state                       401125 non-null int64
ProductGroup                401125 non-null int64
ProductGroupDesc            401125 non-null int64
Enclosure                   401125

In [19]:
# Encoding saledate

#first I'll make a row with only dates
date =[]
for i in data['saledate']:
    date.append(int(i.split('/')[0]))

data['date'] = date

#row with only month
month = []
for i in data['saledate']:
    month.append(int(i.split('/')[1]))

data['month'] = month

#row with only year
year = []
for i in data['saledate']:
    year.append(int(i.split('/')[2][:4]))

data['year'] = year


In [20]:
#since we have extracted the date month and year lets drop the sale date
data = data.drop('saledate',axis=1)

In [21]:
#lastly encoding MachineHoursCurrentMeter
data['MachineHoursCurrentMeter'] = le.fit_transform(data['MachineHoursCurrentMeter'].astype(str))
data.MachineHoursCurrentMeter.value_counts()

15152    258360
0         73126
5266        122
3           116
3375        101
9876         97
13184        90
1598         87
6439         83
2853         83
4245         82
7726         81
7311         76
2259         68
8660         67
6634         65
3835         64
4619         64
844          63
6224         61
14202        60
4946         59
6015         59
5781         58
5265         57
5534         56
6984         55
8661         51
11002        50
2            48
          ...  
1859          1
2970          1
7064          1
15004         1
6808          1
4166          1
6215          1
3394          1
7488          1
4417          1
3829          1
323           1
1436          1
5278          1
5022          1
6720          1
924           1
4766          1
10822         1
412           1
3994          1
2882          1
14747         1
4254          1
2205          1
7232          1
3066          1
14491         1
1947          1
2286          1
Name: MachineHoursCurren

In [22]:
#lastly encoding MachineHoursCurrentMeter
# data['MachineHoursCurrentMeter'] = le.fit_transform(data['MachineHoursCurrentMeter'].astype(str))
data['auctioneerID'] = data['auctioneerID'].fillna(333)
data['auctioneerID'] = data['auctioneerID'].astype(int)
data.auctioneerID.value_counts()

1      185310
2       56440
3       29076
4       20474
333     20136
6       11950
99      11406
7        7846
8        7203
5        7002
10       5790
9        4764
11       3823
12       3418
13       3031
18       2359
14       2277
20       2238
19       2074
16       1792
15       1680
21       1601
22       1429
24       1357
23       1322
17       1275
27       1150
25        959
28        860
26        676
0         407
Name: auctioneerID, dtype: int64

In [23]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 22 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                401125 non-null int64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    401125 non-null int64
fiModelDesc                 401125 non-null int64
fiBaseModel                 401125 non-null int64
fiSecondaryDesc             401125 non-null int64
ProductSize                 401125 non-null int64
fiProductClassDesc          401125 non-null int64
state                       401125 non-null int64
ProductGroup                401125 non-null int64
ProductGroupDesc            401125 non-null int64
Enclosure                   401125 non-null int64
Transmission                401125 non-

In [24]:
data.head(10)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,fiModelDesc,fiBaseModel,fiSecondaryDesc,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Enclosure,Transmission,Hydraulics,date,month,year
0,1139246,66000,999089,3157,121,3,2004,11898,949,295,40,6,58,51,5,5,2,8,0,11,16,2006
1,1139248,57000,117657,77,121,3,1996,9446,1724,526,54,3,61,11,5,5,2,8,0,3,26,2004
2,1139249,10000,434808,7009,121,3,2001,7050,330,109,175,6,38,16,2,2,5,8,3,2,26,2004
3,1139251,38500,1026470,332,121,3,2001,7993,3673,1374,175,5,7,1,3,3,2,8,0,5,19,2011
4,1139253,11000,1057373,17311,121,3,2007,12362,4207,1528,175,6,39,16,2,2,0,8,3,7,23,2009
5,1139255,26500,1001274,4605,121,3,2004,9966,492,174,56,6,1,15,0,0,5,6,12,12,18,2008
6,1139256,21000,772701,1937,121,3,1993,1260,1452,418,47,2,13,0,3,3,0,8,11,8,26,2004
7,1139261,27000,902002,3539,121,3,2001,9738,739,242,40,6,1,9,0,0,5,7,12,11,17,2005
8,1139272,21500,1036251,36003,121,3,2008,7344,778,249,61,4,16,1,3,3,0,8,3,8,27,2009
9,1139275,65000,1016474,3883,121,3,1000,5451,1770,539,20,1,67,0,5,5,2,8,0,8,9,2007


In [25]:
# Lets first save the dataset as its stored in the CPU so that we can use the very same data always.

os.makedirs('tmp', exist_ok=True)
data.to_feather('tmp/raw')

In the future we can simply read it from this fast format.

In [26]:
data_modified = feather.read_dataframe('tmp/raw')

In [27]:
df, y = data_modified.drop('SalePrice',axis=1) , data_modified['SalePrice']

In [28]:
df

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,fiModelDesc,fiBaseModel,fiSecondaryDesc,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Enclosure,Transmission,Hydraulics,date,month,year
0,1139246,999089,3157,121,3,2004,11898,949,295,40,6,58,51,5,5,2,8,0,11,16,2006
1,1139248,117657,77,121,3,1996,9446,1724,526,54,3,61,11,5,5,2,8,0,3,26,2004
2,1139249,434808,7009,121,3,2001,7050,330,109,175,6,38,16,2,2,5,8,3,2,26,2004
3,1139251,1026470,332,121,3,2001,7993,3673,1374,175,5,7,1,3,3,2,8,0,5,19,2011
4,1139253,1057373,17311,121,3,2007,12362,4207,1528,175,6,39,16,2,2,0,8,3,7,23,2009
5,1139255,1001274,4605,121,3,2004,9966,492,174,56,6,1,15,0,0,5,6,12,12,18,2008
6,1139256,772701,1937,121,3,1993,1260,1452,418,47,2,13,0,3,3,0,8,11,8,26,2004
7,1139261,902002,3539,121,3,2001,9738,739,242,40,6,1,9,0,0,5,7,12,11,17,2005
8,1139272,1036251,36003,121,3,2008,7344,778,249,61,4,16,1,3,3,0,8,3,8,27,2009
9,1139275,1016474,3883,121,3,1000,5451,1770,539,20,1,67,0,5,5,2,8,0,8,9,2007


In [29]:
# n_jobs specifies the RandomForestRegressor to create a seperate job for each CPU
m = RandomForestRegressor(n_jobs=-1)
m.fit(df,y)
m.score(df,y)

0.9815858633732859

# Splitting data for validation set and test set.

In [31]:

def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000
n_trn = len(df)-n_valid
modified_train, modified_validation = split_vals(data_modified, n_trn)
X_train, X_validation = split_vals(df, n_trn)
y_train, y_validation = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_validation.shape

((389125, 21), (389125,), (12000, 21))

## Base Model

Let's try our model again, this time with separate training and validation sets.

In [39]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train),rmse(m.predict(X_validation),y_validation),
          m.score(X_train,y_train), m.score(X_validation,y_validation)]
    print (res)

In [40]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train,y_train)
print_score(m)

CPU times: user 52.5 s, sys: 363 ms, total: 52.9 s
Wall time: 15.4 s
[3130.565140367746, 8552.028241548951, 0.9814699369263005, 0.8756186728844428]
